# imports:

In [ ]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import *
from SampleFileFunctions import *
from tunneltoamazondb import getengine
from pandas import concat, read_csv, Series, read_sql

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# generate sample lists:

## currently used lists:

### PNP530list

In [ ]:
folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis_corrected'
PNP530= [f for f in listdir(folder) if isfile(join(folder, f))]
PNP530=[sample.replace('.tsv','') for sample in PNP530]
print len(PNP530)
PNP530[:5]

In [ ]:
PNP530edited=editSampleNamesList(PNP530)
PNP530edited[:20]

In [ ]:
# check whether there are two replicates of the same sample in the list:
for n,sample in enumerate(PNP530):
            if '_' in sample:
                NewName=sample.split('_')[0]
                PNP530[n]=NewName
            else:
                NewName=sample
            if 'b' in NewName:
                NewName=NewName.split('b')[0]
                PNP530[n]=NewName
PNP530[:5]    
    

In [ ]:
print len(PNP530)
print len(set(PNP530))

CONCLUSION: there are no two replicates of the same sample in the list!

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','wb') as fp:
    pickle.dump(PNP530,fp)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
print len(PNP530)
PNP530[:5]

### Cardio126list

In [ ]:
folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/SamplesForAnalysis_corrected'
Cardio126= [f for f in listdir(folder) if isfile(join(folder, f))]
Cardio126=[sample.replace('.tsv','') for sample in Cardio126]
print len(Cardio126)
Cardio126[:5]

In [ ]:
# check whether there are two replicates of the same sample in the list:
for n,sample in enumerate(Cardio126):
            if '_' in sample:
                NewName=sample.split('_')[0]
                Cardio126[n]=NewName
            else:
                NewName=sample
            if 'b' in NewName:
                NewName=NewName.split('b')[0]
                Cardio126[n]=NewName
Cardio126[:5]    
    

In [ ]:
print len(Cardio126)
print len(set(Cardio126))

CONCLUSION: there are no two replicates of the same sample in the list!

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','wb') as fp:
    pickle.dump(Cardio126,fp)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
print len(Cardio126)
Cardio126[:5]

### reseq plate2:

In [ ]:
file_fullPath='%s/Sample files/reseqPlate2samples.xlsx' %MyPath
reseqPlate2=gen_py_BD_list(file_fullPath)
reseqPlate2[:5]

In [ ]:
with open('%s/Sample files/BD lists/reseqPlate2' %MyPath,'wb') as fp:
    pickle.dump(reseqPlate2,fp)

### reseq plate3:

In [ ]:
file_fullPath='%s/Sample files/reseqPlate3samples.xlsx' %MyPath
reseqPlate3=gen_py_BD_list(file_fullPath)
reseqPlate3[:5]

In [ ]:
with open('%s/Sample files/BD lists/reseqPlate3' %MyPath,'wb') as fp:
    pickle.dump(reseqPlate3,fp)

## lists currently not in use:

### plate8:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/Plate8 samples.xlsx'
Plate8samples=pd.read_excel(file1)
Plate8samples['Samples']=Plate8samples['Samples'].astype(str)
Plate8samples['Samples'].dtype

In [ ]:
Plate8samples_list=list(Plate8samples['Samples'])
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Plate8samples_list','wb') as fp:
    pickle.dump(Plate8samples_list,fp)

In [ ]:
print len(Plate8samples_list)

### wait for cleaning

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/samplesToClean_010218.xlsx'
samplesToClean_010218=pd.read_excel(file1)
samplesToClean_010218['DnaID']=samplesToClean_010218['DnaID'].astype(str)
samplesToClean_010218['DnaID'].dtype

In [ ]:
samplesToClean_010218_list=list(samplesToClean_010218['DnaID'])
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/samplesToClean_010218_list','wb') as fp:
    pickle.dump(samplesToClean_010218_list,fp)

### wait for cleaning including the samples with comments:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/samplesToClean_withComments.xlsx'
samplesToClean_withComments=pd.read_excel(file1)

In [ ]:
samplesToClean_withComments.head()

In [ ]:
samplesToClean_withComments_list=list(samplesToClean_withComments['DnaID'])

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/samplesToClean_withComments_list','wb') as fp:
    pickle.dump(samplesToClean_withComments_list,fp)

### cardio list:

In [ ]:
cardioList=["BD950","BD951","BD952","BD953","BD955","BD956","BD957","BD958","BD959","BD962","BD963","BD964","BD965","BD967",
                           "BD971","BD973","BD974","BD978","BD979","BD980","BD982","BD983","BD984","BD985","BD986"]

In [ ]:
print len(cardioList)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/cardioList','wb') as fp:
    pickle.dump(cardioList,fp)

### plate9:

!!!!!Add pnp samples to plate9 list and update all files!!!!

In [ ]:
plate9_file_fullPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/plate9BDs.xlsx'

In [ ]:
plate9_allCardio=gen_py_BD_list(plate9_file_fullPath)

### all PNp samples

In [ ]:
allPNPlist=['BD'+str(x) for x in range(1,951)]

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/allPNPlist','wb') as fp:
    pickle.dump(allPNPlist,fp)

### comment df:

this df is intended to include all sample information I acheive and is not organized well:

In [ ]:
#generate dictionary - ADD SAMPLE:COMMENTS PAIRS HERE###
commentDict={'BD105':'was cleaned in plate3 but from unknown reason was not PCR-ed and sequenced',
            'BD106':'was cleaned in plate3 but from unknown reason was not PCR-ed and sequenced',
            'BD109': 'cleaned in plate3, not enough DNA after cleanup','BD114': 'cleaned in plate3, not enough DNA after cleanup',
            'BD115': 'cleaned in plate3, not enough DNA after cleanup','BD117': 'cleaned in plate3, not enough DNA after cleanup',
            'BD152': 'cleaned in plate3, not enough DNA after cleanup','BD169': 'cleaned in plate3, not enough DNA after cleanup',
            'BD174': 'cleaned in plate3, not enough DNA after cleanup','BD152':'cleaned in plate3, not enough DNA after cleanup',
             'BD155':'cleaned in plate3, not enough DNA after cleanup','BD189':'cleaned in plate3, not enough DNA after cleanup',
             'BD199':'not enough DNA left for cleanup','BD214':'not enough DNA left for cleanup',
             'BD216': 'not enough DNA left for cleanup','BD217':'not enough DNA left for cleanup',
             'BD223': 'not enough DNA after cleanup','BD224':'not enough DNA after cleanup',
             'BD270': 'doesnt appear in the cleanup file', 'BD278': 'doesnt appear in the cleanup file',
             'BD281': 'doesnt appear in the cleanup file','BD287': 'doesnt appear in the cleanup file',
             'BD291': 'doesnt appear in the cleanup file',  'BD296': 'doesnt appear in the cleanup file',
             'BD300': 'doesnt appear in the cleanup file','BD318': 'doesnt appear in the cleanup file',
             'BD320': 'doesnt appear in the cleanup file','BD324': 'doesnt appear in the cleanup file',
             'BD176': 'cleaned in plate3, not enough DNA after cleanup',
             'BD182': 'cleaned in plate3, not enough DNA after cleanup',
             'BD187': 'cleaned in plate3, not enough DNA after cleanup',
            'BD813':'cleaned in plate12, not enough DNA after cleanup','BD814':'cleaned in plate12, not enough DNA after cleanup',
            'BD815':'cleaned in plate12, not enough DNA after cleanup','BD818':'cleaned in plate12, not enough DNA after cleanup',
            'BD819':'cleaned in plate12, not enough DNA after cleanup','BD824':'cleaned in plate12, not enough DNA after cleanup',
            'BD827':'cleaned in plate15, not enough DNA after cleanup','BD853':'cleaned in plate15, not enough DNA after cleanup',
            'BD866':'cleaned in plate12, not enough DNA after cleanup'}

In [ ]:
commentDF=pd.DataFrame({'BD':commentDict.keys(),'Sample Comments':commentDict.values()})

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/commentDF'
commentDF.to_pickle(f1)

### pnp434 list:

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP434samples','rb') as fp:
    PNP434samples=pickle.load(fp)
print len(PNP434samples)
PNP434samples[:5]

In [ ]:
'BD438' in PNP434samples

### pnp490 list:

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP490','rb') as fp:
    PNP490=pickle.load(fp)
print len(PNP490)
PNP490[:5]

### PNP515list

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/BD760.tsv'
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/BD1.tsv'
t1=time.ctime(os.path.getmtime(file1))
t2=time.ctime(os.path.getmtime(file2))
print t1,t2

In [ ]:
t1<t2

In [ ]:
print t1


In [ ]:
folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis'
PNP515= [f for f in listdir(folder) if (isfile(join(folder, f)))&(time.ctime(os.path.getmtime(join(folder, f)))<t1)]
PNP515=[sample.replace('.tsv','') for sample in PNP515]
print len(PNP515)
PNP515[:5]

In [ ]:
PNP515edited=editSampleNamesList(PNP515)
PNP515edited[:20]

In [ ]:
# check whether there are two replicates of the same sample in the list:
for n,sample in enumerate(PNP515):
            if '_' in sample:
                NewName=sample.split('_')[0]
                PNP515[n]=NewName
            else:
                NewName=sample
            if 'b' in NewName:
                NewName=NewName.split('b')[0]
                PNP515[n]=NewName
PNP515[:5]    
    

In [ ]:
print len(PNP515)
print len(set(PNP515))

CONCLUSION: there are no two replicates of the same sample in the list!

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','wb') as fp:
    pickle.dump(PNP515,fp)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)
PNP490[:5]

### Cardio91list

In [ ]:
folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/SamplesForAnalysis'
Cardio91= [f for f in listdir(folder) if isfile(join(folder, f))]
Cardio91=[sample.replace('.tsv','') for sample in Cardio91]
print len(Cardio91)
Cardio91[:5]

In [ ]:
# check whether there are two replicates of the same sample in the list:
for n,sample in enumerate(Cardio91):
            if '_' in sample:
                NewName=sample.split('_')[0]
                Cardio91[n]=NewName
            else:
                NewName=sample
            if 'b' in NewName:
                NewName=NewName.split('b')[0]
                Cardio91[n]=NewName
Cardio91[:5]    
    

In [ ]:
print len(Cardio91)
print len(set(Cardio91))

CONCLUSION: there are no two replicates of the same sample in the list!

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio91','wb') as fp:
    pickle.dump(Cardio91,fp)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio91','rb') as fp:
    Cardio91=pickle.load(fp)
print len(Cardio91)
Cardio91[:5]

### add here more lists if necessary:

# generate table that sort all samples to their curent state:

In [ ]:
from tunneltoamazondb import getengine
from pandas import concat, read_csv, Series, read_sql

In [ ]:
allBDnames = read_sql('select  DnaID from Lab.hostdna', getengine())
allBDnames.head()

In [ ]:
print len(allBDnames)
print allBDnames.nunique()

In [ ]:
#remove None's and nan's:
from numpy import nan
allBDnames_noDups=allBDnames.drop_duplicates()
allBDnames_noDups['DnaID']=allBDnames_noDups['DnaID'].replace('None',np.nan)
allBDnames_noDups=allBDnames_noDups.dropna()
print len(allBDnames_noDups)

In [ ]:
allBDnames_noDups.sort_values(by='DnaID')[-5:]

### generate the table itself:

In [ ]:
# generate the df and its index:
AllBDsamplesSummary=pd.DataFrame(index=allBDnames_noDups['DnaID'])
AllBDsamplesSummary.head()

In [ ]:
#addd columns dictating sample status:
### add new lists if necessary!!###
for n in AllBDsamplesSummary.index:
#     if n in PNP434samples:
#         AllBDsamplesSummary.loc[n,'exist in PNP434']=1
#     else:
#         AllBDsamplesSummary.loc[n,'exist in PNP434']=0
#     if n in PNP490:
#         AllBDsamplesSummary.loc[n,'PNP490']=1
#     else:
#         AllBDsamplesSummary.loc[n,'PNP490']=0
#     if n in PNP515:
#         AllBDsamplesSummary.loc[n,'PNP515']=1
#     else:
#         AllBDsamplesSummary.loc[n,'PNP515']=0
    if n in PNP530:
        AllBDsamplesSummary.loc[n,'PNP530']=1
    else:
        AllBDsamplesSummary.loc[n,'PNP530']=0
#     if n in Cardio91:
#         AllBDsamplesSummary.loc[n,'Cardio91']=1
#     else:
#         AllBDsamplesSummary.loc[n,'Cardio91']=0
    if n in Cardio126:
        AllBDsamplesSummary.loc[n,'Cardio126']=1
    else:
        AllBDsamplesSummary.loc[n,'Cardio126']=0
    if n in reseqPlate2:
        AllBDsamplesSummary.loc[n,'reseqPlate2']=1
    else:
        AllBDsamplesSummary.loc[n,'reseqPlate2']=0
    if n in reseqPlate3:
        AllBDsamplesSummary.loc[n,'reseqPlate3']=1
    else:
        AllBDsamplesSummary.loc[n,'reseqPlate3']=0
#     if n in samplesToResequence_010218_list:
#         AllBDsamplesSummary.loc[n,'awaits resequencing']=1
#     else:
#         AllBDsamplesSummary.loc[n,'awaits resequencing']=0
#     if n in samplesToClean_010218_list:
#         AllBDsamplesSummary.loc[n,'awaits recleaning']=1
#     else:
#         AllBDsamplesSummary.loc[n,'awaits recleaning']=0
#     if n in Plate8samples_list:
#         AllBDsamplesSummary.loc[n,'Plate8']=1
#     else:
#         AllBDsamplesSummary.loc[n,'Plate8']=0
#     if n in cardioList:
#         AllBDsamplesSummary.loc[n,'cardioList']=1
#     else:
#         AllBDsamplesSummary.loc[n,'cardioList']=0
#     if n in plate9_allCardio:
#         AllBDsamplesSummary.loc[n,'plate9_allCardio']=1
#     else:
#         AllBDsamplesSummary.loc[n,'plate9_allCardio']=0

# AllBDsamplesSummary=pd.merge(AllBDsamplesSummary,samplesToClean_withComments,how='left',
#                             left_index=True,right_on='DnaID')

# AllBDsamplesSummary=AllBDsamplesSummary.set_index('DnaID').sort_index(axis=0)

print len(AllBDsamplesSummary)
AllBDsamplesSummary.head(10)
    
    

In [ ]:
AllBDsamplesSummary.tail(20)

In [ ]:
# AllBDsamplesSummary[AllBDsamplesSummary['Comments'].notnull()].head()

In [ ]:
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary.xlsx'
AllBDsamplesSummary.to_excel(file6)

# get studyTypeID for each BD sample:

1. generate a table with UserIDs and an studyTypeID for each - look which users have more than one study and if they have BDs.
2.if there are BDs linked to userIDs with more than one study, assign study manually to the BD. 
3.merge studyID with the total BD summary table and see whether there are missing samples in PNP1
4.summarize everything

5.prepare a summarizing table for the cardio samples


In [ ]:
#load BD summary file
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary.xlsx'
AllBDsamplesSummary=pd.read_excel(file6)
AllBDsamplesSummary.head()

In [ ]:
#load BD-FD-userID table:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_FD_sample_conversion_withSPIDs.xlsx'
BD_FD_sample_conversion_withSPIDs=pd.read_excel(file1)
BD_FD_sample_conversion_withSPIDs.head()

In [ ]:
#generate a table with BD numbers and their respective studyType. if not integer, it means that there 
# were several study types per BD number. probably 1 &2:
BDstudyType=BD_FD_sample_conversion_withSPIDs.groupby('BD').StudyTypeID.mean()
BDstudyType=pd.DataFrame(BDstudyType)
print len(BDstudyType)
BDstudyType.head()

In [ ]:
BDstudyType.tail()

In [ ]:
#merge the summarizing BD table with the studyType:
AllBDsamplesSummary_withStudyType=pd.merge(AllBDsamplesSummary,BDstudyType,how='left',
                                          left_on='DnaID',right_index=True)
AllBDsamplesSummary_withStudyType.head(20)


In [ ]:
AllBDsamplesSummary_withStudyType.groupby('StudyTypeID').DnaID.nunique()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withStudyType.xlsx'
AllBDsamplesSummary_withStudyType.to_excel(file1)

# merge with relationship information:

In [ ]:
#load BD list:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withStudyType.xlsx'
AllBDsamplesSummary_withStudyType=pd.read_excel(file1)
print len(AllBDsamplesSummary_withStudyType)
AllBDsamplesSummary_withStudyType.head()


In [ ]:
#load relationship information:
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/GenRelationsWithBDs_moreBDs3.xlsx'
GenRelationsWithBDs_moreBDs3=pd.read_excel(file6)
print len(GenRelationsWithBDs_moreBDs3)
GenRelationsWithBDs_moreBDs3.head()

### add relationship information to BD list:

In [ ]:
#add relationship information to BD list:
AllBDsamplesSummary_withRelationInfo=pd.merge(AllBDsamplesSummary_withStudyType,GenRelationsWithBDs_moreBDs3,how='left',
                                             left_on='DnaID',right_on='DnaID')
print len(AllBDsamplesSummary_withRelationInfo)
AllBDsamplesSummary_withRelationInfo.head()


In [ ]:
#drop duplicate BDs:
AllBDsamplesSummary_withRelationInfo2=AllBDsamplesSummary_withRelationInfo.drop_duplicates(subset=['DnaID'])
print len(AllBDsamplesSummary_withRelationInfo2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withRelationInfo.xlsx'
AllBDsamplesSummary_withRelationInfo2.to_excel(file1)

### add BD status to relationship info:

In [ ]:
GenRelationsWithBDs_SampleStatus=pd.merge(GenRelationsWithBDs_moreBDs3,AllBDsamplesSummary_withStudyType,how='left',
                                             left_on='DnaID',right_on='DnaID')
print len(GenRelationsWithBDs_SampleStatus)
GenRelationsWithBDs_SampleStatus.head()

In [ ]:
#drop duplicate BDs:
GenRelationsWithBDs_SampleStatus2=GenRelationsWithBDs_SampleStatus.drop_duplicates(subset=['DnaID','correctRegNum'])
print len(GenRelationsWithBDs_SampleStatus2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/GenRelationsWithBDs_SampleStatus.xlsx'
GenRelationsWithBDs_SampleStatus2.to_excel(file1)

# merge with comments DF:

In [ ]:
#load BD file:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withRelationInfo.xlsx'
AllBDsamplesSummary_withRelationInfo2=pd.read_excel(file1)

In [ ]:
#load comments:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/commentDF'
commentDF=pd.read_pickle(file1)

In [ ]:
AllBDsamplesSummary_withRelation_withComments=pd.merge(AllBDsamplesSummary_withRelationInfo2,commentDF,
                                                      how='left',left_on='DnaID', right_on='BD')
AllBDsamplesSummary_withRelation_withComments.head()

In [ ]:
AllBDsamplesSummary_withRelation_withComments[AllBDsamplesSummary_withRelation_withComments['Sample Comments'].notnull()]
AllBDsamplesSummary_withRelation_withComments=AllBDsamplesSummary_withRelation_withComments.drop('BD', axis=1)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withRelation_withComments.xlsx'
AllBDsamplesSummary_withRelation_withComments.to_excel(file1)

# generate comprehensive PNP BD summary
inclusing wet lab information and sequencing information

### generate df with all existing BD numbers:

In [ ]:
#check duplicated BD numbers:
allBDnamesCheck = read_sql('select  * from Lab.hostdna', getengine())
allBD_duplicated=allBDnamesCheck[allBDnamesCheck.duplicated(subset='DnaID',keep=False)]
allBD_duplicated

In [ ]:
#check which BD is the last in the database:
allBDnames = read_sql('select  DnaID from Lab.hostdna', getengine())
allBDnames['index']=allBDnames['DnaID'].str.replace('BD','')
allBDnames['index']=allBDnames['index'].astype('float64')
allBDnames=allBDnames.sort_values(by='index',ascending=False)
print len(allBDnames)
allBDnames.head()

# (the length of DB is longer than the number of BDs due to several 'None' rows in the db.
# see former cell)

In [ ]:
#generate list of all BDs from 1 to 1099:
BDlist=['BD'+str(x) for x in range(1,1100)]

In [ ]:
#generate df from this list:
BDdfIndex=pd.DataFrame({'BD':BDlist,'BDindex':range(1,1100)})
print len(BDdfIndex)
BDdfIndex.head()

In [ ]:
#add 'study type'indication:
BDdfIndex['Study']=np.where(BDdfIndex['BDindex']<950,'PNP','Cardio')

### add wetlab summary information:

In [ ]:
#load existing wet lab summary:
f1='/net/mraid08/export/genie/Lab/ImmunologyProject/BD samples summary files/CleanUpAndPCRplateSummary.xlsx'
CleanUpAndPCRplateSummary=pd.read_excel(f1)
print len(CleanUpAndPCRplateSummary)
CleanUpAndPCRplateSummary.head()

In [ ]:
# check for complete duplicates:
CleanUpAndPCRplateSummary[CleanUpAndPCRplateSummary.duplicated()]

In [ ]:
#drop nan rows:
CleanUpAndPCRplateSummary=CleanUpAndPCRplateSummary[CleanUpAndPCRplateSummary['BD'].notnull()]
print len(CleanUpAndPCRplateSummary)

In [ ]:
#look for samples that appear more than once in the file:
print len(CleanUpAndPCRplateSummary[CleanUpAndPCRplateSummary.duplicated(subset='BD',keep=False)].sort_values(by='BDindex'))
CleanUpAndPCRplateSummary[CleanUpAndPCRplateSummary.duplicated(subset='BD',keep=False)].sort_values(by='BDindex')

In [ ]:
#add indication for existing in this file:
CleanUpAndPCRplateSummary['inOriginalFile']=1

In [ ]:
#merge the existing file with the list of all BDs from 1 to 1099, to find BDs that do not appear in this list:
#use how=outer to enable samples that were cleaned more than once. 
allBDnumbersWetLab=pd.merge(BDdfIndex,CleanUpAndPCRplateSummary,how='outer',left_on='BD',right_on='BD')
print allBDnumbersWetLab.head()
allBDnumbersWetLab=allBDnumbersWetLab.sort_values(by='BDindex_x')

In [ ]:
print len(BDdfIndex)
print len(CleanUpAndPCRplateSummary)
print CleanUpAndPCRplateSummary['BD'].nunique()
print len(allBDnumbersWetLab)

In [ ]:
#how many duplicated rows in the merged file:
print len(allBDnumbersWetLab[allBDnumbersWetLab.duplicated(subset='BD',keep=False)].sort_values(by='BDindex_x'))
#no duplicate were added during merging!

### add cleaning indications, comments and generate need-to-clean file:

In [ ]:
#add indications for samples that were cleaned or not
allBDnumbersWetLab['Cleaned']=np.where(allBDnumbersWetLab['CleanUp plate'].notnull(),1,0)
allBDnumbersWetLab.head()

In [ ]:
# add comments why not cleaned from Michal's file: ###check if there is a newer file to use here!
f1='/net/mraid08/export/genie/Lab/ImmunologyProject/BD samples summary files/notCleaned_08042018.xlsx'
notCleaned_08042018=pd.read_excel(f1)
notCleaned_08042018.head()

In [ ]:
#merge reason for not cleaning into the main file:
print len(allBDnumbersWetLab)
notCleaned_small=notCleaned_08042018[['BD','Why not cleaned?']]
allBDnumbersWetLab=pd.merge(allBDnumbersWetLab,notCleaned_small,how='left', 
                           left_on='BD', right_on='BD')
print len(allBDnumbersWetLab)
allBDnumbersWetLab.head()

In [ ]:
notCleaned=allBDnumbersWetLab[allBDnumbersWetLab['Cleaned']==0]
print len(notCleaned)

In [ ]:
#save samples that were not cleaned:
f1='/net/mraid08/export/genie/Lab/ImmunologyProject/BD samples summary files/notCleaned_%s.xlsx' %cdate
notCleaned.to_excel(f1)

In [ ]:
allBDnumbersWetLab[allBDnumbersWetLab['BD']=='BD109']

### generate PCR indications and lists:

In [ ]:
#add indications for samples that were taken for PCR:
allBDnumbersWetLab['takenForPCR']=np.where(allBDnumbersWetLab['PCR Plate'].notnull(),1,0)
allBDnumbersWetLab.head()

In [ ]:
#test how many samples were cleaned but not taken for PCR:
print len(allBDnumbersWetLab[(allBDnumbersWetLab['Cleaned']==1)&(allBDnumbersWetLab['takenForPCR']==0)])
# currently 292 samples

In [ ]:
#check how many and which samples were clean with resulting concentratio of above 27ng/ul and were not taken for PCR:

notTakenEnoughConc=allBDnumbersWetLab[(allBDnumbersWetLab['Cleaned']==1)&(allBDnumbersWetLab['takenForPCR']==0)&(allBDnumbersWetLab['ng/ul after cleanUp']>27)]
print len(notTakenEnoughConc)
notTakenEnoughConc['BD'].unique()


In [ ]:
#check if any of this sample has duplicate that was taken:
needToPCR=[]
for sample in list(notTakenEnoughConc['BD']):
    sampleDF=allBDnumbersWetLab[allBDnumbersWetLab['BD']==sample]
    PCR_DF=sampleDF[sampleDF['PCR Plate'].notnull()]
#     print sample, len(sampleDF),len(PCR_DF)
    if len(PCR_DF)==0:
        print sample
        needToPCR.append(sample)
allBDnumbersWetLab['needToPCR']=np.where(allBDnumbersWetLab['BD'].isin(needToPCR),1,0)

In [ ]:
needToPCRdf=allBDnumbersWetLab[allBDnumbersWetLab['needToPCR']==1]
needToPCRdf

In [ ]:
#save need to PCR samples:
f1='/net/mraid08/export/genie/Lab/ImmunologyProject/BD samples summary files/needToPCRdf_%s.xlsx' %cdate
needToPCRdf.to_excel(f1)

### merge with sequencing information

In [ ]:
#load sequencing information:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/AllBDsamplesSummary_withRelation_withComments.xlsx'
AllBDsamplesSummary_withComments=pd.read_excel(file1)
AllBDsamplesSummary_withComments.head()


In [ ]:
#take only relevant columns:
AllBDsamplesSummary_withComments_small=AllBDsamplesSummary_withComments[['DnaID','PNP530',
                                                                         'Cardio126','reseqPlate2',
                                                                         'reseqPlate3','StudyTypeID','Relations',
                                                                         'Sample Comments']]

### merge wet lab and sequencing lab info:

In [ ]:

allBDnumbersWetAndSeqInfo=pd.merge(allBDnumbersWetLab,AllBDsamplesSummary_withComments_small,how='left',
                                  left_on='BD',right_on='DnaID')
allBDnumbersWetAndSeqInfo.head()


In [ ]:
print len(allBDnumbersWetLab)
print len(AllBDsamplesSummary_withComments_small)
print len(allBDnumbersWetAndSeqInfo)

# check which samples are not in pnp530 or awaits reseq but were PCRed:

In [ ]:

allBDnumbersWetAndSeqInfo['inSeqPipeline']=allBDnumbersWetAndSeqInfo['PNP530']+\
allBDnumbersWetAndSeqInfo['reseqPlate2']+allBDnumbersWetAndSeqInfo['reseqPlate3']
allBDnumbersWetAndSeqInfo.head()

In [ ]:
PNPnotFinishedPipeline=allBDnumbersWetAndSeqInfo[(allBDnumbersWetAndSeqInfo['inSeqPipeline']\
==0)&(allBDnumbersWetAndSeqInfo['takenForPCR']==1)&(allBDnumbersWetAndSeqInfo['BDindex_x']<950)]

In [ ]:
PNPnotFinishedPipeline['PCR Plate'].value_counts()

In [ ]:
PNPnotFinishedPipeline[['BD','PCR Plate','PNP530','reseqPlate2','reseqPlate3','inSeqPipeline']]

### add analysis comments for problematic samples

In [ ]:
for n in PNPnotFinishedPipeline.index:
    allBDnumbersWetAndSeqInfo.loc[n,'data analysis comments']='sequenced and analyzed but\
    not included in cohort due to data problems'
    

In [ ]:
allBDnumbersWetAndSeqInfo.tail()

### clean file:

In [ ]:
# correct '#rows per BD' column:

allBDnumbersWetAndSeqInfo['#rows per BD'] = allBDnumbersWetAndSeqInfo['BD']\
.apply(lambda x: allBDnumbersWetAndSeqInfo['BD'].value_counts()[x])

# for n in allBDnumbersWetAndSeqInfo.index:
#     BD=allBDnumbersWetAndSeqInfo.loc[n,'BD']
#     rowsPerBD=
#     allBDnumbersWetAndSeqInfo['#rows per BD']

In [ ]:
allBDnumbersWetAndSeqInfo.columns.values

In [ ]:
columns_to_keep=['BD', 'BDindex_x', 'Study_x','#rows per BD','reg num',
        'DNA extraction kit','Cleaned', 'Why not cleaned?','CleanUp plate', 
        'Well in CleanUp Plate', 'CleanUpKit', u'ng/ul after cleanUp',
        'Comments during cleanUp', 'Is more than 31ng/ul?','take for PCR',
        'takenForPCR', 'needToPCR','PCR Plate','PCRComments', 'sequencing comments',
        'inOriginalFile', 'PNP530','Cardio126','reseqPlate2', 'reseqPlate3',
        'StudyTypeID', 'Relations', 'Sample Comments', 'inSeqPipeline',
       'data analysis comments']

In [ ]:
allBDnumbersWetAndSeqInfo=allBDnumbersWetAndSeqInfo[columns_to_keep]

In [ ]:
allBDnumbersWetAndSeqInfo=allBDnumbersWetAndSeqInfo.sort_values(by='BDindex_x')

### save combined file:

In [ ]:

f1='/net/mraid08/export/genie/Lab/ImmunologyProject/BD samples summary files/allBDnumbersWetAndSeqInfo_%s.xlsx' %cdate
allBDnumbersWetAndSeqInfo.to_excel(f1)


# get some 'suspected pairs' for Noa

In [ ]:
sample1List=['BD146','BD481','BD113','BD481','BD328','BD328','BD788','BD505','BD431','BD699','BD788','BD431']
sample2List=['BD145','BD314','BD317','BD800','BD319','BD453','BD245','BD547','BD440','BD706','BD808','BD440']

SuspectedPairs=pd.DataFrame({'sample1BD':sample1List,'sample2BD': sample2List})
SuspectedPairs


In [ ]:
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_date_correctReg_userID'
BD_date_correctReg_userID=pd.read_pickle(file6)
BD_date_correctReg_userID.head()

In [ ]:
SuspectedPairs_withRegNums=pd.merge(SuspectedPairs,BD_date_correctReg_userID[['correct registration code','DnaID']],how='left',
                                   left_on='sample1BD',right_on='DnaID')
SuspectedPairs_withRegNums=SuspectedPairs_withRegNums.drop('DnaID',axis=1)
SuspectedPairs_withRegNums=SuspectedPairs_withRegNums.rename(columns={'correct registration code':'sample1RegNum'})
SuspectedPairs_withRegNums=pd.merge(SuspectedPairs_withRegNums,BD_date_correctReg_userID[['correct registration code','DnaID']],how='left',
                                   left_on='sample2BD',right_on='DnaID')
SuspectedPairs_withRegNums=SuspectedPairs_withRegNums.drop('DnaID',axis=1)
SuspectedPairs_withRegNums=SuspectedPairs_withRegNums.rename(columns={'correct registration code':'sample2RegNum'})
SuspectedPairs_withRegNums.head()

In [ ]:
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/SuspectedPairs_withRegNums.xlsx'
SuspectedPairs_withRegNums.to_excel(file6)

# look for cardio samples that are not in plate8 or plate9:

In [ ]:
df=AllBDsamplesSummary_withStudyType
df['id']=df['DnaID'].str.replace('BD','').astype(int)
df2=df[(df['id']>949)&(df['Plate8']==0)&(df['plate9_allCardio']==0)]
print len(df2)

In [ ]:
cardio_remaining=list(df2['DnaID'].sort_values())
cardio_remaining

# develop: generate pythonic sample name list (string) from excel file

In [ ]:
l=['3849571BD985','3137757BD986','3068162BD987']
l


In [ ]:
regex_BD=re.compile('..BD')

In [ ]:
regex_BD

In [ ]:
new=['BD'+x.split('BD')[1] for x in l if re.search(regex_BD,x)]
new

In [ ]:
## this function was copied to 'SampleFileFunctions.py' - use and update there!!##

def gen_py_BD_list(file_fullPath):
    BDdf=pd.read_excel(file_fullPath)
    BDlist=list(BDdf.iloc[:,0])
    regex_BD=re.compile('..BD')
    BDlist_clean=['BD'+x.split('BD')[1] for x in BDlist if re.search(regex_BD,x)]
    
    return BDlist_clean
    

In [ ]:
file_fullPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/plate9BDs.xlsx'

In [ ]:
BDlist_clean=gen_py_BD_list(file_fullPath)

In [ ]:
BDlist_clean[2]